In [1]:
#use the intrinio api python wrapper to gather the data
from __future__ import print_function
import time
import intrinio_sdk 
from intrinio_sdk.rest import ApiException
from pprint import pprint
import pandas as pd
import numpy as np
import mysql.connector
import math
import datetime as dt
import sys
import importlib as ib
import GetData as gd

In [2]:
#save and load function
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
start_date_test = '1990-01-01' # date | Get historical date on or before this date (optional)
end_date_test = '2019-07-31'
ticker_test = "MSFT"
tags = ['close_price','open_price','high_price','low_price','adj_volume','adjdilutedeps','dividend']

In [7]:
#reload after changing the script
ib.reload(gd)

<module 'GetData' from 'C:\\Users\\Zijun\\Documents\\Projects\\Volatility Prediction\\GetData.py'>

In [8]:
result_test = gd.get_data(start_date_test,end_date_test,tags,ticker_test,90)

Exception when calling SecurityApi->get_security_historical_data: (502)
Reason: Bad Gateway
HTTP response headers: HTTPHeaderDict({'Server': 'awselb/2.0', 'Date': 'Fri, 13 Dec 2019 21:18:40 GMT', 'Content-Type': 'text/html', 'Content-Length': '138', 'Connection': 'keep-alive'})
HTTP response body: <html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>





In [14]:
#check length
len(result_test['high_price'])/252

29.99206349206349

In [16]:
result_test['close_price'][-1]

{'date': datetime.date(1990, 1, 2), 'value': 88.75}

In [17]:
#save the result
save_obj(result_test,"pkl_data/result_intraday_30yr")

In [28]:
result_test = load_obj("pkl_data/result_intraday_30yr")

In [29]:
price_tags = ['close_price','open_price','high_price','low_price','adj_volume']

In [30]:
#merge all the prices and volumes into one dataframe

def merge_data(tags,input_data):
    #first tag
    result = {}
    result['date'] = [x['date']for x in input_data[tags[0]]]
    result[tags[0]] = [x['value'] for x in input_data[tags[0]]]
    result_df = pd.DataFrame(result)
    for t in tags[1:]:
        result_tmp = {}
        result_tmp['date'] = [x['date'] for x in input_data[t]]
        result_tmp[t] = [x['value'] for x in input_data[t]]
        result_tmp_df = pd.DataFrame(result_tmp)
        #merge
        result_df = pd.merge(result_df, result_tmp_df , left_on='date', right_on='date')
    return result_df    

In [31]:
result_df = merge_data(price_tags,result_test)

In [32]:
result_df

,date,close_price,open_price,high_price,low_price,adj_volume
0,2019-12-12,153.24,151.65,153.44,151.020,24645366.0
1,2019-12-11,151.70,151.54,151.87,150.330,18860001.0
2,2019-12-10,151.13,151.29,151.89,150.765,16481060.0
3,2019-12-09,151.36,151.07,152.21,150.910,16741350.0
4,2019-12-06,151.75,150.99,151.87,150.270,16410400.0
...,...,...,...,...,...,...
7553,1990-01-08,91.00,89.50,91.00,88.500,58982400.0
7554,1990-01-05,89.62,91.50,92.00,89.500,69564816.0
7555,1990-01-04,91.87,89.25,92.00,88.750,125740800.0
7556,1990-01-03,89.25,89.50,90.25,88.500,113772816.0


In [33]:
#sort it ascendingly
result_df = result_df.sort_values(by = ['date'])
result_df.reset_index(inplace = True)
result_df.drop(columns=['index'])
result_df

,index,date,close_price,open_price,high_price,low_price,adj_volume
0,7557,1990-01-02,88.75,87.25,88.75,86.120,53033616.0
1,7556,1990-01-03,89.25,89.50,90.25,88.500,113772816.0
2,7555,1990-01-04,91.87,89.25,92.00,88.750,125740800.0
3,7554,1990-01-05,89.62,91.50,92.00,89.500,69564816.0
4,7553,1990-01-08,91.00,89.50,91.00,88.500,58982400.0
...,...,...,...,...,...,...,...
7553,4,2019-12-06,151.75,150.99,151.87,150.270,16410400.0
7554,3,2019-12-09,151.36,151.07,152.21,150.910,16741350.0
7555,2,2019-12-10,151.13,151.29,151.89,150.765,16481060.0
7556,1,2019-12-11,151.70,151.54,151.87,150.330,18860001.0


In [34]:
#save
save_obj(result_df,"pkl_data/result_test_30yr_df")

In [35]:
#load
result_df = load_obj("pkl_data/result_test_30yr_df")

In [37]:
#now add the other columns for quarterly data:'netdebttoebitda','ltdebttoebitda','adjdilutedeps','dividend'
tags_loop = ['adjdilutedeps','dividend']

for t in tags_loop:
    #populate that column with nan
    result_df[t] = np.nan
    
    result_tmp = {}
    result_tmp['date'] = [x['date'] for x in result_test[t]][::-1]
    result_tmp[t] = [x['value'] for x in result_test[t]][::-1]
    
    
    #loop through the dataframe and set the value in between the quarterly announcement as the prev one
    for index,row in result_df.iterrows():
        date_in_df = row['date']
        #take the previous value for the quarterly data
        #result_tmp['date'] should be sorted
        for dd in result_tmp['date']:
            if dd <= date_in_df:
                index_tmp = result_tmp['date'].index(dd)
                value_tmp = result_tmp[t][index_tmp]
                result_df.loc[index,t] = value_tmp
                #early exit
                continue

In [38]:
result_df

,index,date,close_price,open_price,high_price,low_price,adj_volume,adjdilutedeps,dividend
0,7557,1990-01-02,88.75,87.25,88.75,86.120,53033616.0,NaN,NaN
1,7556,1990-01-03,89.25,89.50,90.25,88.500,113772816.0,NaN,NaN
2,7555,1990-01-04,91.87,89.25,92.00,88.750,125740800.0,NaN,NaN
3,7554,1990-01-05,89.62,91.50,92.00,89.500,69564816.0,NaN,NaN
4,7553,1990-01-08,91.00,89.50,91.00,88.500,58982400.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7553,4,2019-12-06,151.75,150.99,151.87,150.270,16410400.0,5.06,0.51
7554,3,2019-12-09,151.36,151.07,152.21,150.910,16741350.0,5.06,0.51
7555,2,2019-12-10,151.13,151.29,151.89,150.765,16481060.0,5.06,0.51
7556,1,2019-12-11,151.70,151.54,151.87,150.330,18860001.0,5.06,0.51


In [39]:
result_df.drop(columns=['index'],inplace = True)

In [40]:
result_df

,date,close_price,open_price,high_price,low_price,adj_volume,adjdilutedeps,dividend
0,1990-01-02,88.75,87.25,88.75,86.120,53033616.0,NaN,NaN
1,1990-01-03,89.25,89.50,90.25,88.500,113772816.0,NaN,NaN
2,1990-01-04,91.87,89.25,92.00,88.750,125740800.0,NaN,NaN
3,1990-01-05,89.62,91.50,92.00,89.500,69564816.0,NaN,NaN
4,1990-01-08,91.00,89.50,91.00,88.500,58982400.0,NaN,NaN
...,...,...,...,...,...,...,...,...
7553,2019-12-06,151.75,150.99,151.87,150.270,16410400.0,5.06,0.51
7554,2019-12-09,151.36,151.07,152.21,150.910,16741350.0,5.06,0.51
7555,2019-12-10,151.13,151.29,151.89,150.765,16481060.0,5.06,0.51
7556,2019-12-11,151.70,151.54,151.87,150.330,18860001.0,5.06,0.51


In [41]:
save_obj(result_df,"pkl_data/result_test_30yr_df_final")